In [2]:
#libraries
from geopy.geocoders import Yandex
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date as dt
from tqdm import tqdm
from pathlib import Path
import time
import warnings
warnings.filterwarnings("ignore")


In [3]:
#da eseguire solo la prima volta per installare la libreria
#!pip install pathlib

In [4]:
def find_streets(h):
    h = h[['Latitude', 'Longitude']]
    j = 0
    passed = []
    
    for i in range(len(h)):
        if (i % 70) == 0:
            print(i)
        try:
            location = geolocator.reverse((h.loc[i][0], h.loc[i][1]))

        except:
            print(j, 'WAIT')
            time.sleep(3)
            location = geolocator.reverse((h.loc[i][0], h.loc[i][1]))

        try:
                passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['DependentLocality']['DependentLocality']['Thoroughfare']['ThoroughfareName'])
        except:

            try:
                    passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['Thoroughfare']['ThoroughfareName'])
            except: 

                try:
                        passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['DependentLocality']['Thoroughfare']['ThoroughfareName'])
                except:
                        passed.append(None)

                #location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['Thoroughfare']['ThoroughfareName']
                #print(i,'hi')
                #passed.append(j)
        j+=1
    print(passed)
    return passed

In [5]:

geolocator = Yandex(user_agent="my-application", lang="en_US")
big_android = pd.DataFrame()
j = 724
pathlist = Path('').glob('**/*gps.csv')
for path in pathlist:
    j += 1
    
    #import files
    file = str(path)
    print(file, j)
    h = pd.read_csv(r'./Android files/' + str(725) + '_track_2_0_gps.csv', skiprows=1) #gps

    try:
        r = pd.read_csv(r'./Android files/' + str(725) +'_sj.csv') #science_journal
    except FileNotFoundError as fnf_error:
        print(fnf_error)
        break

        
    #order/filter column
    col = []
    for i in h.columns:
        i = i.replace(' ', '')
        col.append(i)
    h.columns = col
    h['street'] = find_streets(h)

    h = h.filter(['Time','Latitude','Longitude','Elevation','Speed','street'])
    
    #to datetime
    h['Time'] = '2019-04-25 '+ h['Time']
    h['Time'] = pd.to_datetime(h.Time, format='%Y-%m-%d %H:%M:%S')
    
    #adjust Speed
    h['Speed'] = h.Speed*4
    
    #put n. person, type of car
    h['N_person'] = int(file.split('_')[2])
    h['Car_type'] = int(file.split('_')[3])
    
    #create diff
    data1=h.iloc[0]['Time']
    h['Diff']=(h['Time']-data1).dt.seconds
    h['Diff']=h['Diff']*1000
    h = h.drop_duplicates(subset=['Diff'], keep='first')
    
    #assign a reference to sj_file
    l=[str(i) for i in h.index]
    gap=[h.iloc[i]['Diff'] for i in range(len(h))]
    del l[-1]
    try:
        r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))#,duplicates='drop'))
    except:
        r['relative_time'] = r['timestamp'].diff()
        r['relative_time'][0] = 0
        r.drop(['timestamp'], axis =1)
        r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))#,duplicates='drop'))

    r['tag']=r['tag'].fillna(value='0')
    
    #adjust sj
    sj_new=r.filter(['relative_time','AccZ','tag'])
    sj_new.columns=['Time','Acc','Reference']
    sj_new['Reference'] = pd.to_numeric(sj_new['Reference'])
    
    #merged df
    df=pd.merge(sj_new,h,left_on='Reference',right_on=h.index)
    df=df.drop(columns=['Reference','Diff'])
    df.columns=['Elapse(ms)','Acc','Date','Latitude','Longitude','Elevation','Speed(Km/h)','N_person','Car_type', 'Street']
    df=df.reindex(['Date','Elapse(ms)','Latitude','Longitude', 'Street', 'Elevation','Speed(Km/h)','Acc','N_person','Car_type'],axis=1)
    df=df[df['Speed(Km/h)']>1]
    

    break

    #append current df to previuos ones
    if len(big_android)==0:
        big_android = df
    else:
        big_android = big_android.append(df) 
print('hi')
big_android.to_csv('All_Records_Android.csv', index=False)

Android files\701_track_4_1_gps.csv 725
0
34 WAIT
70
['Via Camilla', 'Via Camilla', 'Via Camilla', None, None, 'Via Camilla', None, 'Via Camilla', 'Via Camilla', 'Via Camilla', 'Via Camilla', None, None, 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via R. de Cesare', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'Via Clelia', 'V

In [11]:
df

,Date,Elapse(ms),Latitude,Longitude,Street,Elevation,Speed(Km/h),Acc,N_person,Car_type
1240,2019-04-25 08:01:21,2016,41.874026,12.523953,1,67.381202,2.836607,10.160992,Via Camilla,4
1241,2019-04-25 08:01:21,2020,41.874026,12.523953,1,67.381202,2.836607,10.419566,Via Camilla,4
1242,2019-04-25 08:01:21,2051,41.874026,12.523953,1,67.381202,2.836607,10.496181,Via Camilla,4
1243,2019-04-25 08:01:21,2086,41.874026,12.523953,1,67.381202,2.836607,10.093954,Via Camilla,4
1244,2019-04-25 08:01:21,2090,41.874026,12.523953,1,67.381202,2.836607,9.797073,Via Camilla,4
1245,2019-04-25 08:01:21,2115,41.874026,12.523953,1,67.381202,2.836607,9.624691,Via Camilla,4
1246,2019-04-25 08:01:21,2120,41.874026,12.523953,1,67.381202,2.836607,9.375694,Via Camilla,4
1247,2019-04-25 08:01:21,2151,41.874026,12.523953,1,67.381202,2.836607,9.481039,Via Camilla,4
1248,2019-04-25 08:01:21,2182,41.874026,12.523953,1,67.381202,2.836607,9.078813,Via Camilla,4
1249,2019-04-25 08:01:21,2190,41.874026,12.523953,1,67.381202,2.836607,9.059659,Via Camilla,4


In [10]:
h

,Time,Latitude,Longitude,Elevation,Speed,street,N_person,Car_type,Diff
0,2019-04-25 08:01:19,41.873820,12.523813,49.790001,0.000000,Via Camilla,4,1,0
1,2019-04-25 08:01:21,41.874026,12.523953,67.381202,2.836607,Via Camilla,4,1,2000
2,2019-04-25 08:01:22,41.874037,12.523974,82.288352,4.776311,Via Camilla,4,1,3000
3,2019-04-25 08:01:23,41.873819,12.523878,99.069230,5.149247,None,4,1,4000
4,2019-04-25 08:01:24,41.873732,12.523759,97.567819,4.868631,None,4,1,5000
5,2019-04-25 08:01:25,41.873950,12.523875,118.422288,2.759723,Via Camilla,4,1,6000
6,2019-04-25 08:01:26,41.873730,12.523816,108.014318,0.000000,None,4,1,7000
7,2019-04-25 08:01:39,41.873819,12.523785,92.701463,0.000000,Via Camilla,4,1,20000
8,2019-04-25 08:01:59,41.873995,12.523667,57.128657,0.000000,Via Camilla,4,1,40000
9,2019-04-25 08:02:19,41.873897,12.523859,28.932739,0.000000,Via Camilla,4,1,60000


In [28]:
for i in range(1,len(r.timestamp)):
    r.loc[i] = int(r.loc[i][0] - r.loc[i-1][0] )

KeyboardInterrupt: 

In [35]:
r.head()

,timestamp,AccZ
0,1555950352913,9.279925e+00
1,-1555950352910,-1.555950e+12
2,3111900705860,3.111901e+12
3,-3111900705842,-3.111901e+12
4,4667851058812,4.667851e+12


In [53]:
r['relative_time'] = r['timestamp'].diff()
r['relative_time'][0] = 0
r.drop(['timestamp'], axis =1)

C:\Users\mccol\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,AccZ,relative_time
0,9.279925,0.0
1,9.423578,3.0
2,9.739613,37.0
3,9.366117,15.0
4,9.356541,20.0
5,9.586384,19.0
6,9.835381,20.0
7,9.519346,20.0
8,9.471462,23.0
9,9.567230,17.0


In [50]:
r['relative_time'][0] =0

0         0.0
1         3.0
2        37.0
3        15.0
4        20.0
5        19.0
6        20.0
7        20.0
8        23.0
9        17.0
10       20.0
11       20.0
12       20.0
13       21.0
14       19.0
15       20.0
16       20.0
17       23.0
18       17.0
19       20.0
20       20.0
21       20.0
22       21.0
23       19.0
24       20.0
25       21.0
26       19.0
27       21.0
28       20.0
29       19.0
         ... 
60762    20.0
60763    20.0
60764    20.0
60765    20.0
60766    20.0
60767    20.0
60768    20.0
60769    20.0
60770    21.0
60771    19.0
60772    20.0
60773    20.0
60774    20.0
60775    20.0
60776    20.0
60777    20.0
60778    20.0
60779    21.0
60780    19.0
60781    20.0
60782    20.0
60783    20.0
60784    20.0
60785    20.0
60786    21.0
60787    19.0
60788    20.0
60789    20.0
60790    20.0
60791    20.0
Name: relative_time, Length: 60792, dtype: float64